# Modulo 3: Ejercicio 5
## Alumno: Jose Suarez

### 1. Crea un DataFrame con datos de ventas por producto y región. Utiliza multi-índices para representar la información por año y trimestre. Luego, calcula la suma de ventas por trimestre para cada producto.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

delimitador = "\n" + "-" * 60 + "\n"

clientes = pd.read_csv("clientes.csv", encoding='latin-1')
productos = pd.read_csv("productos.csv", encoding='latin-1')
boletas = pd.read_csv("boletas.csv", encoding='latin-1')
boletas["fecha"] = pd.to_datetime(boletas['fecha'])
detalle = pd.read_csv("detalle.csv", encoding='latin-1')

# print((clientes.dtypes), end= delimitador)
# print((productos), end= delimitador)
# print((boletas), end= delimitador)
# print((detalle.dtypes), end= delimitador)

    codigo            nombre  precio
0      101             Arroz    1500
1      102            Fideos    1200
2      103  Aceite de cocina   18000
3      104            Harina    2000
4      105             Leche    3000
5      106            Azúcar    2500
6      107              Café    5000
7      108               Pan    1500
8      109          Galletas    3500
9      110       Mantequilla    2800
10     111             Queso    6000
11     112             Yogur    2000
12     113      Agua mineral    1200
13     114             Jugos    4000
14     115    Frutas frescas    7000
15     116          Verduras    5500
16     117    Carne de pollo    4800
17     118      Carne de res   10000
18     119           Pescado    9000
19     120            Helado    2500
------------------------------------------------------------


In [2]:
# Se crea dataframe al hacer merge de las tablas y se borran columnas inecesarias.
detalle_productos = detalle.merge(productos, left_on = "codigo_producto", 
                                  right_on = "codigo", how="left").drop("codigo", axis=1)

detalle_productos["Subtotal"] = detalle_productos["cantidad"] * detalle_productos["precio"]

boleta_productos = boletas.merge(detalle_productos, left_on = "numero", 
                                 right_on = "numero_boleta", how="right").drop("numero", axis=1)

clientes_productos = clientes[["rut","comuna"]].merge(boleta_productos, left_on = "rut", 
                                    right_on="rut_cliente", 
                                    how="right").drop(["rut_cliente", "rut"], axis=1)
clientes_productos = clientes_productos.reindex(["numero_boleta","fecha","comuna","codigo_producto",
                                                 "nombre","cantidad", "Subtotal"],axis=1)

print(clientes_productos.groupby(["comuna","nombre"]).sum("subtotal"))

                               numero_boleta  codigo_producto  cantidad  \
comuna       nombre                                                       
Antofagasta  Café                         11              107         4   
             Fideos                       21              102         5   
             Galletas                     11              109         7   
             Harina                       22              104         1   
             Helado                       21              120         4   
             Pescado                      16              119         4   
Concepción   Fideos                        2              102         5   
             Harina                        2              104         1   
Iquique      Agua mineral                 14              113         3   
             Azúcar                       19              106         7   
             Harina                       19              104         3   
             Verduras    

In [8]:
clientes_productos["año"] = clientes_productos["fecha"].dt.year
clientes_productos["trimestre"] = clientes_productos["fecha"].dt.quarter

print(clientes_productos.groupby(["año", "trimestre"]).sum("subtotal").drop([
    "numero_boleta","codigo_producto"], axis=1), end=delimitador)

print(clientes_productos.groupby(["año", "trimestre","nombre",
                                  "codigo_producto"]).sum(["subtotal"])
                                    .drop(["numero_boleta"],axis =1), end=delimitador)

                cantidad  Subtotal
año  trimestre                    
2023 4               187    923100
------------------------------------------------------------
                                                 cantidad  Subtotal
año  trimestre nombre           codigo_producto                    
2023 4         Aceite de cocina 103                    11    198000
               Agua mineral     113                     6      7200
               Arroz            101                    24     36000
               Azúcar           106                    12     30000
               Café             107                    13     65000
               Carne de pollo   117                     6     28800
               Fideos           102                    12     14400
               Frutas frescas   115                     2     14000
               Galletas         109                     7     24500
               Harina           104                     9     18000
               Hel

### 2. Tienes dos DataFrames: uno con información de clientes (ID, nombre, dirección) y otro con sus pedidos (ID del cliente, producto, cantidad). Realiza una combinación de datos para obtener un DataFrame que muestre el nombre del cliente, su dirección y los productos pedidos por cada cliente.

In [4]:
np.random.seed(0)

idClientes = np.random.randint(1000,3000,size=5)
nombreClientes = ["Roberto C.", "Roberta R.", "Tony M.", "Cuack C.", "Condor I.T."]

direccionClientes = ["Comuna " + str(np.random.randint((_*10),(_*10+10))) for _ in range(5)]

clientesDF = pd.DataFrame({
    "id": idClientes,
    "nombre cliente": nombreClientes,
    "comuna": direccionClientes
})

productos = ["Producto " + str(np.random.randint((_*5),(_*5+5))) for _ in range(10)]

indices = pd.MultiIndex.from_product([idClientes, productos], names=['id', 'producto'])
cantidades = np.random.randint(1, 100, size=(len(indices)))

pedidosDF = pd.DataFrame({"cantidad": cantidades}, index = indices).reset_index()

pedidos_clientes = pedidosDF.merge(clientesDF,left_on="id", right_on="id").set_index(
    "id").reindex(["nombre cliente","comuna","producto","cantidad"],axis=1)
print(pedidos_clientes)


     nombre cliente     comuna     producto  cantidad
id                                                   
1684     Roberto C.   Comuna 3   Producto 2        78
1684     Roberto C.   Comuna 3   Producto 9        73
1684     Roberto C.   Comuna 3  Producto 10        10
1684     Roberto C.   Comuna 3  Producto 15        21
1684     Roberto C.   Comuna 3  Producto 24        81
1684     Roberto C.   Comuna 3  Producto 27        70
1684     Roberto C.   Comuna 3  Producto 31        80
1684     Roberto C.   Comuna 3  Producto 35        48
1684     Roberto C.   Comuna 3  Producto 41        65
1684     Roberto C.   Comuna 3  Producto 46        83
1559     Roberta R.  Comuna 17   Producto 2        89
1559     Roberta R.  Comuna 17   Producto 9        50
1559     Roberta R.  Comuna 17  Producto 10        30
1559     Roberta R.  Comuna 17  Producto 15        20
1559     Roberta R.  Comuna 17  Producto 24        20
1559     Roberta R.  Comuna 17  Producto 27        15
1559     Roberta R.  Comuna 

### 3. Una Tabla de BINGO
Para una tarde de bingo se usarán tablas generadas con las
siguientes reglas:

    ● Los 5 números en cada columna son aleatorios ordenados no repetidos de acuerdo al rango asignado por columna.
    ● La casilla central de la tabla tiene valor marcado cero o ‘libre’. Realice un programa para generar una tabla de bingo

In [9]:
def GeneradorNumerosSinRep(cotaInferior, CotaSuperior, tamaño):
    listaARetornar = []
    for num in range(0, tamaño):
        while True:
            generado = np.random.randint(cotaInferior, CotaSuperior + 1)
            if generado in listaARetornar:
                pass
            else:
                listaARetornar.append(generado)
                break
                
    listaARetornar.sort()
    return listaARetornar

def GenerarCarton(idCarton):
    cotasNumeros = [[15 * _ + 1, 15 * _ + 15, 5] for _ in range(5)]

    bingo = []
    for rangos in cotasNumeros:
        bingo.append(GeneradorNumerosSinRep(rangos[0], rangos[1], rangos[2]))

    bingo[2][2] = "libre"

    for letra in "BINGO":
        print(letra, end= " "*(5-len(letra)))
    print(idCarton,end = "\n"+"-"* 23+"\n")

    for columna in bingo:
        for celda in columna:
            print(celda, end=" "*(5-len(str(celda))))
        print()
    print("-"* 23)
    
cartonesAGenerar = int(input("Cuantos jugaran hoy vilma?"))


for carton in range(1,cartonesAGenerar + 1):
    GenerarCarton(carton)


Cuantos jugaran hoy vilma?5
B    I    N    G    O    1
-----------------------
5    6    7    10   12   
18   23   24   25   29   
37   39   libre42   44   
47   52   54   58   59   
63   64   69   71   73   
-----------------------
B    I    N    G    O    2
-----------------------
4    6    7    14   15   
16   20   23   24   27   
36   37   libre42   44   
48   49   54   57   58   
64   66   68   70   75   
-----------------------
B    I    N    G    O    3
-----------------------
4    5    6    8    14   
19   23   25   26   30   
33   34   libre41   43   
47   51   53   57   59   
62   63   66   69   73   
-----------------------
B    I    N    G    O    4
-----------------------
1    3    5    9    12   
16   21   24   29   30   
31   32   libre39   42   
46   49   50   54   55   
63   64   68   73   75   
-----------------------
B    I    N    G    O    5
-----------------------
2    3    5    6    14   
18   21   27   28   29   
36   37   libre42   45   
47   48   52   53   58 